In [1]:
import pandas as pd
import numpy as np
import time, os

## Init

In [2]:
df_test = pd.read_csv('./data/ejemplo_de_solucion.csv')
df_train = pd.read_csv('./data/postulaciones/postulaciones_train.csv')

In [3]:
df_aDetalle = pd.read_csv('./data/avisos/avisos_detalle.csv')
df_aOnline = pd.read_csv('./data/avisos/avisos_online.csv')
df_postEdu = pd.read_csv('./data/postulantes/postulantes_educacion.csv')
df_postGenero = pd.read_csv('./data/postulantes/postulantes_genero_edad.csv')

In [4]:
df_aDetalle.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,1,VENDEDOR/A PROVINCIA DE SANTA FE,<p><strong><strong>Empresa:</strong></strong> ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,1,Enfermeras,<p>Solicitamos para importante cadena de farma...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste
2,1000150677,1,Chofer de taxi,<p>TE GUSTA MANEJAR? QUERES GANAR PLATA HACIEN...,Capital Federal,NaN,Empedrado 2336,Full-time,Senior / Semi-Senior,Transporte,FAMITAX SRL
3,1000610287,1,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,<p><strong>Somos una empresa multinacional que...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A
4,1000872556,1,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL


In [5]:
df_aOnline.head()

,idaviso,online_desde,online_hasta
0,8725750,2018-01-15,2018-02-10
1,12543760,2018-01-18,2018-03-20
2,12812680,2018-02-20,2018-03-21
3,17903700,2018-03-20,2018-04-17
4,1000132160,2018-01-17,2018-01-31


In [7]:
df_postEdu.head()

,idpostulante,nombre,estado
0,NdJl,Posgrado,En Curso
1,8BkL,Universitario,En Curso
2,1d2B,Universitario,En Curso
3,NPBx,Universitario,En Curso
4,NPBx,Master,En Curso


In [8]:
df_postGenero.head()

,idpostulante,fechanacimiento,sexo
0,NM5M,1970-12-03,FEM
1,5awk,1962-12-04,FEM
2,ZaO5,1978-08-10,FEM
3,NdJl,1969-05-09,MASC
4,eo2p,1981-02-16,MASC


In [9]:
df_train.head()

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34
1,1111920714,NM5M,2018-02-06 09:04:50
2,1112346945,NM5M,2018-02-22 09:04:47
3,1112345547,NM5M,2018-02-22 09:04:59
4,1112237522,5awk,2018-01-25 18:55:03


In [10]:
df_test.head()

,idaviso,idpostulante
0,1112427970,ZaO5
1,1112427970,NdJl
2,1112427970,5kNq
3,1112427970,Ez8J
4,1112427970,erL9


In [11]:
len(df_test.idpostulante.unique())


156232

In [12]:
df_test.loc[df_test['idpostulante']=='ZaO5']

,idaviso,idpostulante
0,1112427970,ZaO5
156232,1112094756,ZaO5


In [24]:
#Elimino filas repetidas
df_postEdu=df_postEdu.drop_duplicates(subset='idpostulante')
df_postGenero=df_postGenero.drop_duplicates(subset='idpostulante')

In [25]:
# Acumulo informacion de los postulantes
df_postulante = df_postGenero.merge(df_postEdu, on='idpostulante')
df_postulante

,idpostulante,fechanacimiento,sexo,nombre,estado
0,NM5M,1970-12-03,FEM,Secundario,Graduado
1,5awk,1962-12-04,FEM,Universitario,Graduado
2,ZaO5,1978-08-10,FEM,Terciario/Técnico,Graduado
3,NdJl,1969-05-09,MASC,Posgrado,En Curso
4,eo2p,1981-02-16,MASC,Secundario,Graduado
...,...,...,...,...,...
447904,GNZr6xw,1997-08-17,MASC,Universitario,En Curso
447905,OqrJ8zv,1998-06-23,FEM,Terciario/Técnico,Graduado
447906,X95VzpK,1998-02-25,MASC,Universitario,En Curso
447907,QNr9A2N,1992-08-01,MASC,Master,En Curso


,idpostulante,nombre,estado
671,5awk,Universitario,Graduado


In [32]:
print(df_aDetalle.shape, df_aOnline.shape)

(24950, 11) (18362, 3)


In [47]:
# Me quedo con las publicaciones que tiene fecha de inicio y fin, las otras las elimino.
df_publicacion = df_aDetalle.merge(df_aOnline, on='idaviso')
# Elimino idPais: Son todos 1
df_publicacion = df_publicacion.drop(['idpais','descripcion','tipo_de_trabajo', 'ciudad', 'mapacalle'], axis=1)

In [48]:
df_publicacion

,idaviso,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,online_desde,online_hasta
0,8725750,VENDEDOR/A PROVINCIA DE SANTA FE,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Comercial,VENTOR,2018-01-15,2018-02-10
1,17903700,Enfermeras,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste,2018-03-20,2018-04-17
2,1000610287,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A,2018-01-15,2018-03-17
3,1001135716,Vendedor Viajante TUCUMAN/SANTIAGO DEL ESTERO,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Ventas,Wurth Argentina S.A,2018-01-15,2018-04-16
4,1001326344,Vendedor Viajante RECONQUISTA/AVELLANEDA,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Ventas,Wurth Argentina S.A,2018-01-15,2018-03-27
...,...,...,...,...,...,...,...,...,...
18357,1112407528,Analista Contable - Impositivo,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Contabilidad,GI GROUP Argentina,2018-03-16,2018-03-18
18358,1112407400,Entrevista CAJERO REPOSITOR presentarse LUNES ...,Gran Buenos Aires,Full-time,Junior,Comercial,Suministra,2018-03-16,2018-03-20
18359,1112407609,Tecnico sector moldes,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Laboratorio,GAES S A,2018-03-16,2018-04-13
18360,1112407882,Analista de Nominas,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Administración de Personal,Nexo Consultora,2018-03-19,2018-03-25
